In [1]:
import pyaudio
import wave
import subprocess
import sounddevice
import sys
import time
import os

from pydub import AudioSegment


import os
import _pickle as cPickle
import numpy as np
from scipy.io.wavfile import read
from FeatureExtraction import extract_features
#from speakerfeatures import extract_features
import warnings
warnings.filterwarnings("ignore")
import time
import sys


C:\Users\Sherif\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:

same_output_file = False


CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 1.1
record_len = RECORD_SECONDS
wav_path = "output.wav"
record = True
vad = True
normalize = True
recognize = True

condition = True
counter = 0
frames = []
while condition and counter < 700 :
    # condition = False
    counter += 1
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)
    # print("Started recording...", end="")

    if record:
        # if frames:
            # frames = frames[len(frames)//2:]
            # record_len = RECORD_SECONDS - 1
        frames = []
        data_bytes = b''
        for i in range(0, int(RATE / CHUNK * record_len)):
            data = stream.read(CHUNK)
            frames.append(data)
            data_bytes += data


        stream.stop_stream()
        stream.close()
        p.terminate()
        # print(type(data_bytes))
        # sys.exit()
        if same_output_file:
            ext = ""
        else: 
            ext = "./recordings_dump/" + str(counter)
        wf = wave.open(ext+wav_path, 'wb')
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
        wf.close()
        # print("Done recording! - F")
        # print(len(frames))

    # if vad:
    #     pass

    if normalize:
        with wave.open(ext+wav_path, 'rb') as wav_file:
            # Get the audio file properties
            sample_width = wav_file.getsampwidth()
            num_channels = wav_file.getnchannels()
            sample_rate = wav_file.getframerate()
            num_frames = wav_file.getnframes()

            # Read the audio data
            audio_data = wav_file.readframes(num_frames)


        # Convert the audio data to AudioSegment object
        audio = AudioSegment(
            data=audio_data,
            sample_width=sample_width,
            frame_rate=sample_rate,
            channels=num_channels
        )

        # Normalize the volume
        normalized_audio = audio.normalize()

        # Change the sampling rate to 1
        normalized_audio = normalized_audio.set_frame_rate(16000)

        # Combine all channels to one channel
        normalized_audio = normalized_audio.set_channels(1)

        # Export the normalized audio as WAV file
        normalized_audio.export(ext+wav_path.split(".")[0]+"_normalized.wav", format='wav')
        path = ext+wav_path.split(".")[0]+"_normalized.wav"


    if recognize:
        #path where training speakers will be saved
        modelpath = "models/"

        gmm_files = [os.path.join(modelpath,fname) for fname in 
                    os.listdir(modelpath) if fname.endswith('.gmm')]

        #Load the Gaussian Models
        models    = [cPickle.load(open(fname,'rb')) for fname in gmm_files]
        # print("Loaded Models: ", len(models))
        speakers   = [fname.split("/")[-1].split(".gmm")[0] for fname 
                    in gmm_files]

        # path = wav_path.split(".")[0]+"_normalized.wav"
        path = ext+wav_path
        # print("Testing Audio : ", path)
        time1 = time.time()   
        sr,audio = read(path)
        vector   = extract_features(audio,sr)
        log_likelihood = np.zeros(len(models)) 
        for i in range(len(models)):
            gmm    = models[i]  #checking with each model one by one
            scores = np.array(gmm.score(vector))
            log_likelihood[i] = scores.sum()
        winner=np.argmax(log_likelihood)
        winner_score = log_likelihood[winner]
        predicted_speaker = speakers[winner]
        sample_time = time.time() - time1
        # print(f"{predicted_speaker}", log_likelihood)
        print(f"{counter}_{predicted_speaker}")
        # print("Time taken in ms: ", sample_time*1000)
    # time.sleep(1)


1_Silence_1
2_Omar_a1
3_Silence_1
4_Silence_1
5_Silence_1
6_Sherif_a1
7_Sherif_e1
8_Silence_1
9_Silence_1
10_Sherif_a1
11_Silence_1
12_Silence_1
13_Sherif_a1
14_Silence_1
15_Silence_1
16_Silence_1
17_Youssef_e1
18_Silence_1
19_Sherif_e1
20_Silence_1
21_Sherif_e1
22_Sherif_e1
23_Silence_1
24_Silence_1
25_Sherif_a1
26_Silence_1
27_Silence_1
28_Silence_1
29_Sherif_a1
30_Reem_a1
31_Renad_a1
32_Sherif_e1
33_Silence_1
34_Renad_e1
35_Sherif_e1
36_Silence_1
37_Silence_1
38_Sherif_a1
39_Silence_1
40_Silence_1
41_Omar_a1
42_Reem_e1
43_Omar_a1
44_Sherif_a1
45_Reem_e1
46_Omar_a1
47_Sherif_e1
48_Abdelrahman Said_e1
49_Sherif_a1
50_Youssef_e1
51_Youssef_e1
52_Youssef_e1
53_Youssef_e1
54_Omar_e1
55_Sherif_a1
56_Youssef_a1
57_Silence_1
58_Omar_a1
59_Abdelrahman Said_e1
60_Abdelrahman Said_e1
61_Silence_1
62_Sherif_e1
63_Sherif_e1
64_Sherif_e1
65_Sherif_e1
66_Renad_e1
67_Reem_e1
68_Sherif_a1
69_Silence_1
70_Silence_1
71_Silence_1
72_Sherif_e1
73_Silence_1
74_Silence_1
75_Sherif_a1
76_Omar_a1
77_Silence

KeyboardInterrupt: 

In [48]:

source   = "2-wav_testing/"   

same_output_file = False


CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 3.1
record_len = RECORD_SECONDS
wav_path = "output.wav"
record = True
vad = True
normalize = True
recognize = True

condition = True
counter = 5
frames = []
while condition and counter < 10 :
    # condition = False
    counter += 1
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)
    print("Started recording...", end="")

    if record:
        if frames:
            frames = frames[len(frames)//3:]
            record_len = RECORD_SECONDS - 2
        data_bytes = b''
        for i in range(0, int(RATE / CHUNK * record_len)):
            data = stream.read(CHUNK)
            frames.append(data)
            data_bytes += data


        stream.stop_stream()
        stream.close()
        p.terminate()
        # print(type(data_bytes))
        # sys.exit()
        if same_output_file:
            ext = ""
        else: 
            ext = "./recordings_dump/" + str(counter)
        wf = wave.open(ext+wav_path, 'wb')
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
        wf.close()
        print("Done recording! - F")
        # print(len(frames))

    # if vad:
    #     pass

    if normalize:
        with wave.open(ext+wav_path, 'rb') as wav_file:
            # Get the audio file properties
            sample_width = wav_file.getsampwidth()
            num_channels = wav_file.getnchannels()
            sample_rate = wav_file.getframerate()
            num_frames = wav_file.getnframes()

            # Read the audio data
            audio_data = wav_file.readframes(num_frames)


        # Convert the audio data to AudioSegment object
        audio = AudioSegment(
            data=audio_data,
            sample_width=sample_width,
            frame_rate=sample_rate,
            channels=num_channels
        )

        # Normalize the volume
        normalized_audio = audio.normalize()

        # Change the sampling rate to 1
        normalized_audio = normalized_audio.set_frame_rate(16000)

        # Combine all channels to one channel
        normalized_audio = normalized_audio.set_channels(1)

        # Export the normalized audio as WAV file
        normalized_audio.export(ext+wav_path.split(".")[0]+"_normalized.wav", format='wav')
        path = ext+wav_path.split(".")[0]+"_normalized.wav"


    if recognize:
        #path where training speakers will be saved
        modelpath = "models/"

        gmm_files = [os.path.join(modelpath,fname) for fname in 
                    os.listdir(modelpath) if fname.endswith('.gmm')]

        #Load the Gaussian Models
        models    = [cPickle.load(open(fname,'rb')) for fname in gmm_files]
        # print("Loaded Models: ", len(models))
        speakers   = [fname.split("/")[-1].split(".gmm")[0] for fname 
                    in gmm_files]

        # path = wav_path.split(".")[0]+"_normalized.wav"
        path = ext+wav_path
        # print("Testing Audio : ", path)
        time1 = time.time()   
        sr,audio = read(path)
        vector   = extract_features(audio,sr)
        log_likelihood = np.zeros(len(models)) 
        for i in range(len(models)):
            gmm    = models[i]  #checking with each model one by one
            scores = np.array(gmm.score(vector))
            log_likelihood[i] = scores.sum()
        winner=np.argmax(log_likelihood)
        winner_score = log_likelihood[winner]
        predicted_speaker = speakers[winner]
        sample_time = time.time() - time1
        print(f"{predicted_speaker}", log_likelihood)
        # print("Time taken in ms: ", sample_time*1000)
    # time.sleep(1)


Started recording...Done recording! - F
Reem_a1 [-32.76447426 -32.30079641 -31.10354531 -31.91071523 -31.79944575
 -32.35427014]
Started recording...Done recording! - F
Reem_a1 [-32.56040529 -31.92070615 -31.52778457 -32.07814506 -31.91837925
 -32.29736347]
Started recording...Done recording! - F
Reem_a1 [-32.7011745  -32.31892179 -31.67203436 -32.12703129 -32.38901516
 -32.54915354]
Started recording...Done recording! - F
Reem_a1 [-32.59340194 -32.03538906 -31.70798461 -31.93584483 -32.28995086
 -32.17485973]
Started recording...Done recording! - F
Sherif_a1 [-31.51790128 -32.02687426 -31.42536591 -31.43124396 -31.09650497
 -31.38635346]


In [ ]:
"""
silence:
Sherif_a1 [-32.27361522 -30.88630014 -31.88865575 -33.01585415 -27.92731392
 -30.4037003 ]
Started recording...Done recording! - F
Sherif_a1 [-32.27361522 -30.88630014 -31.88865575 -33.01585415 -27.92731392
 -30.4037003 ]
Started recording...Done recording! - F
Sherif_a1 [-32.27361522 -30.88630014 -31.88865575 -33.01585415 -27.92731392
 -30.4037003 ]
Started recording...Done recording! - F
Sherif_a1 [-32.27361522 -30.88630014 -31.88865575 -33.01585415 -27.92731392
 -30.4037003 ]
Started recording...Done recording! - F
Sherif_a1 [-32.27361522 -30.88630014 -31.88865575 -33.01585415 -27.92731392
 -30.4037003 ]
Started recording...Done recording! - F
Sherif_a1 [-32.27361522 -30.88630014 -31.88865575 -33.01585415 -27.92731392
 -30.4037003 ]

 speech:
 Sherif_a1 [-32.27361522 -30.88630014 -31.88865575 -33.01585415 -27.92731392
 -30.4037003 ]
Started recording...Done recording! - F
Sherif_a1 [-32.27361522 -30.88630014 -31.88865575 -33.01585415 -27.92731392
 -30.4037003 ]
Started recording...Done recording! - F
Sherif_a1 [-32.27361522 -30.88630014 -31.88865575 -33.01585415 -27.92731392
 -30.4037003 ]
Started recording...Done recording! - F
Sherif_a1 [-32.27361522 -30.88630014 -31.88865575 -33.01585415 -27.92731392
 -30.4037003 ]
Started recording...Done recording! - F
Sherif_a1 [-32.27361522 -30.88630014 -31.88865575 -33.01585415 -27.92731392
 -30.4037003 ]
"""